In [58]:
import openml
from openml.tasks import list_tasks, TaskType
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from src import GibbsForest
import logging


# Load the OpenML dataset
regression_tasks = list_tasks(task_type = TaskType.SUPERVISED_REGRESSION)
small_tasks_ids = []
for task_id, task_value in regression_tasks.items():
    """We want datasets with instances between 5000 and 10000, no missing values, and no symbolic features"""
    if ('NumberOfInstances' in task_value.keys() and task_value['NumberOfInstances'] < 10000 and task_value['NumberOfInstances'] > 5000 and
    'NumberOfMissingValues' in task_value.keys() and task_value['NumberOfMissingValues'] == 0 and 
    'NumberOfSymbolicFeatures' in task_value.keys() and task_value['NumberOfSymbolicFeatures'] == 0):
        small_tasks_ids.append(task_id)

In [59]:
logging.getLogger("openml.extensions.sklearn.extension").setLevel(logging.ERROR)
if not hasattr(GibbsForest, '__version__'):
    GibbsForest.__version__ = "0.0.1"  # Use an appropriate version number

print(len(small_tasks_ids))

88


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from src import Losses

task_id = small_tasks_ids[20]
task = openml.tasks.get_task(task_id)
X, y = task.get_X_and_y()
print(f"Y mean: {y.mean():.4f}")

gibbs_params = {"eta": 0.1,
            "feature_subsample": 0.6,
            "max_depth": 5,
            "min_samples": 2,
            "n_trees": 80, 
            'row_subsample': 0.9, 
            'warmup_depth': 2, 
            'loss_fn': Losses.LeastSquaresLoss(), 
            'reg_lambda': 0,
            'reg_gamma': 1}

dyna = GibbsForest(**gibbs_params)

xgb_params = {
    'eta': 0.1, 
    'subsample': 0.99, 
    'max_depth': 5, 
    'reg_lambda': 0.1, 
    'gamma': 1
}

xgb = XGBRegressor(**xgb_params)

"""rf_params = {
            "max_depth": 5,
            "max_features": 0.50,
            "min_samples_leaf": 3,
            "n_estimators": 68}
dyna = RandomForestRegressor(**rf_params)"""

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
dyna.fit(X_train, y_train)
train_error = mean_absolute_error(y_train, dyna.predict(X_train))
test_error = mean_absolute_error(y_test, dyna.predict(X_test))
print(f"Train error: {train_error:.4f}")
print(f"Test error: {test_error:.4f}")
"""booster = dyna.get_booster()
df = booster.trees_to_dataframe()
total_leaves = (df["Feature"] == "Leaf").sum()
print(f"Total leaves: {total_leaves}")"""

Y mean: 83.9689


In [50]:
import numpy as np
errors = np.array([[1.55012961e-01, 1.19690567e-01, 1.18715062e-01],
 [2.27864006e+00, 1.63303653e+00, 1.59774050e+00],
 [1.11279170e-03, 1.30388785e-03, 1.12549638e-03],
 [2.61028864e+00, 2.56309775e+00, 2.53819391e+00],
 [2.54181707e+00, 2.28245487e+00, 1.87539849e+00],
 [5.17212119e-01, 4.22303067e-01, 4.00673763e-01],
 [5.00497202e-01, 4.19176071e-01, 3.75706172e-01],
 [4.87178472e-01, 4.36459287e-01, 3.70631891e-01],
 [5.61965561e-01, 5.18073543e-01, 4.96628538e-01],
 [1.07452048e-01, 1.08739641e-01, 1.08610724e-01],
 [8.42492473e-03, 9.02710288e-03, 8.74388075e-03],
 [1.85544239e-02, 1.86710448e-02, 1.86979661e-02],
 [1.93822924e-02, 1.93739274e-02, 1.93643104e-02],
 [2.66070075e+00, 2.47643256e+00, 2.43504228e+00],
 [1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
 [5.87860977e-02, 5.53437466e-02, 5.47524916e-02],
 [2.52429177e+00, 2.10119247e+00, 1.90582350e+00],
 [2.96629201e-02, 2.45982695e-02, 2.26927044e-02],
 [2.37716708e+00, 1.73599986e+00, 1.56691770e+00],
 [1.53521432e-01, 1.14978454e-01, 1.09412079e-01],
 [2.42959334e+00, 1.68032178e+00, 1.69157321e+00],
 [1.06947566e-03, 1.27401975e-03, 1.12249952e-03],
 [2.55821045e+00, 2.54205453e+00, 2.56533770e+00],
 [2.48475628e+00, 2.07870051e+00, 1.90476740e+00],
 [5.29935374e-01, 5.12014280e-01, 4.84926766e-01],
 [1.09046146e-01, 1.09390472e-01, 1.11111156e-01],
 [8.44670889e-03, 8.59229324e-03, 7.76171153e-03],
 [1.98315069e-02, 1.99174567e-02, 1.98971291e-02],
 [1.99302548e-02, 1.99821859e-02, 2.00077061e-02],
 [2.70376614e+00, 2.35912366e+00, 2.37085021e+00],
 [1.00000000e+02, 1.00000000e+02, 1.00000000e+02],
 [5.73799461e-02, 5.55275076e-02, 5.38979819e-02],
 [2.62285960e+00, 2.23544209e+00, 1.90958221e+00],
 [3.00306642e-02, 2.62155545e-02, 2.24895160e-02],
 [2.32018394e+00, 1.70108601e+00, 1.57216678e+00],
 [4.70959173e-01, 4.07630021e-01, 3.63760006e-01],
 [5.66143161e-02, 6.20849066e-02, 1.00000000e+02],
 [3.11922659e+00, 3.08294092e+00, 3.08291001e+00],
 [4.50745914e-02, 6.04226148e-02, 1.00000000e+02],
 [3.15928103e+00, 3.17913887e+00, 3.12251508e+00],
 [3.13860185e+00, 3.13832430e+00, 3.12889946e+00],
 [5.28808006e-01, 4.35402776e-01, 4.02457457e-01],
 [5.56639091e-02, 7.52744360e-02, 1.00000000e+02],
 [3.04484814e+00, 3.06971813e+00, 2.98879607e+00],
 [3.11096512e+00, 3.11384177e+00, 3.09488298e+00],
 [4.99300625e-01, 5.08102605e-01, 3.83251740e-01],
 [4.17841828e-02, 6.20872682e-02, 1.00000000e+02],
 [3.08642248e+00, 3.13621224e+00, 3.04740250e+00],
 [3.06899134e+00, 3.05935289e+00, 3.04960891e+00],
 [4.98806938e-01, 4.38883170e-01, 3.69288010e-01]])

relative_gibbs_error = [
   error[2] / error[0] for error in errors 
]
print(np.where(errors[:, 2] == 100))

error_array = errors[errors[:, 2] != 100]

print(f"Errors for Random forest: {np.mean(error_array[:, 0])}")
print(f"Errors for XGBoost: {np.mean(error_array[:, 1])}")
print(f"Errors for Gibbs Forest: {np.mean(error_array[:, 2])}")

print(f"Fraction of datasets Gibbs Forest is better than Random Forest: {np.mean(error_array[:, 2] < error_array[:, 0])}")
print(f"Fraction of datasets Gibbs Forest is better than XGBoost: {np.mean(error_array[:, 2] < error_array[:, 1])}")

print(f"Average relative improvement of Gibbs Forest over Random Forest: {np.mean((error_array[:, 0] - error_array[:, 2]) / error_array[:, 0])}")
print(f"Average relative improvement of Gibbs Forest over XGBoost: {np.mean((error_array[:, 1] - error_array[:, 2]) / error_array[:, 1])}")    

(array([14, 30, 36, 38, 42, 46]),)
Errors for Random forest: 1.3007481757745454
Errors for XGBoost: 1.1813441520618182
Errors for Gibbs Forest: 1.1316849013131818
Fraction of datasets Gibbs Forest is better than Random Forest: 0.7954545454545454
Fraction of datasets Gibbs Forest is better than XGBoost: 0.8636363636363636
Average relative improvement of Gibbs Forest over Random Forest: 0.1256627863698241
Average relative improvement of Gibbs Forest over XGBoost: 0.05585898940360959


In [4]:
trees = dyna._trees
predictions = [tree.predict(X_test) for tree in trees]
tree_biases = np.mean(predictions - y_test, axis = 1)
correlations = np.zeros((len(trees), len(trees)))
for i in range(len(trees)):
    for j in range(i, len(trees)):
        correlations[i, j] = np.corrcoef(predictions[i], predictions[j])[0, 1]
        correlations[j, i] = correlations[i, j]
print(f"Correlations: {np.mean(correlations)}")
print(f"Tree biases: {np.mean(tree_biases)}")

NameError: name 'np' is not defined

In [ ]:
trees = dyna.estimators_
predictions = [tree.predict(X_test) for tree in trees]
tree_biases = np.mean(predictions - y_test, axis = 1)
correlations = np.zeros((len(trees), len(trees)))
for i in range(len(trees)):
    for j in range(i, len(trees)):
        correlations[i, j] = np.corrcoef(predictions[i], predictions[j])[0, 1]
        correlations[j, i] = correlations[i, j]
print(f"Correlations: {np.mean(correlations)}")
print(f"Tree biases: {np.mean(tree_biases)}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error

# -------------------------------
# Assumptions:
#   - dyna: your trained random forest model (e.g., RandomForestRegressor)
#           with an attribute `estimators_` (list of individual tree models)
#   - X_test: test features (numpynp.array or similar)
#   - y_test: true test targets (numpynp.array)
#
# Replace the following dummy definitions with your actual data/model.
# -------------------------------
# Example (comment these out if you already have these defined):
# from sklearn.datasets import make_regression
# from sklearn.ensemble import RandomForestRegressor
#
# X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# dyna = RandomForestRegressor(n_estimators=50, random_state=42)
# dyna.fit(X_train, y_train)

# -------------------------------
# 1. Compute Predictions for Each Tree
# -------------------------------
trees = dyna._trees
n_trees = len(trees)
# Each tree produces predictions on the test set; shape: (n_trees, n_samples)
predictions = np.array([tree.predict(X_test) for tree in trees])

# Ensemble prediction is the average over trees:
ensemble_pred = np.mean(predictions, axis=0)

# -------------------------------
# 2. Compute Residuals (Errors)
# -------------------------------
# Residuals for each tree and for the ensemble:
residuals = predictions - y_test  # shape: (n_trees, n_samples)
ensemble_residual = ensemble_pred - y_test

# -------------------------------
# 3. Correlation Among Tree Predictions
# -------------------------------
# Compute the correlation matrix among the trees’ predictions.
correlation_matrix = np.corrcoef(predictions, rowvar=False)
# Compute the mean correlation (using only the off-diagonal values)
upper_tri_indices = np.triu_indices(n_trees, k=1)
mean_correlation = np.mean(correlation_matrix[upper_tri_indices])
print("Mean correlation among tree predictions: {:.4f}".format(mean_correlation))

# -------------------------------
# 4. Correlation Among Tree Residuals
# -------------------------------
resid_corr_matrix = np.corrcoef(residuals)
mean_resid_corr = np.mean(resid_corr_matrix[upper_tri_indices])
print("Mean correlation among tree residuals: {:.4f}".format(mean_resid_corr))

# -------------------------------
# 5. Compute MSE and MAE for Each Tree and the Ensemble
# -------------------------------
tree_mse = np.array([mean_squared_error(y_test, predictions[i]) for i in range(n_trees)])
tree_mae = np.array([mean_absolute_error(y_test, predictions[i]) for i in range(n_trees)])
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
ensemble_mae = mean_absolute_error(y_test, ensemble_pred)

print("Average individual tree MSE: {:.4f}".format(np.mean(tree_mse)))
print("Ensemble MSE: {:.4f}".format(ensemble_mse))
print("Average individual tree MAE: {:.4f}".format(np.mean(tree_mae)))
print("Ensemble MAE: {:.4f}".format(ensemble_mae))

# -------------------------------
# 6. Compute Prediction Variance Across Trees (Per Sample)
# -------------------------------
# For each test sample, compute variance among the tree predictions
variance_per_sample = np.var(predictions, axis=0)
avg_prediction_variance = np.mean(variance_per_sample)
print("Average variance of tree predictions across samples: {:.4f}".format(avg_prediction_variance))

# -------------------------------
# 7. Ambiguity Decomposition (Regression)
# -------------------------------
# One way to look at ensemble benefits is via the ambiguity decomposition:
#   Ensemble MSE = Average tree MSE - Ambiguity
ambiguity = np.mean(tree_mse) - ensemble_mse
print("Ambiguity (Average tree MSE - Ensemble MSE): {:.4f}".format(ambiguity))

# -------------------------------
# 8. Feature Importances (If Available)
# -------------------------------
# If your individual trees have a `feature_importances_` attribute,
# compute the average importance for each feature.
try:
    feature_importances = np.array([tree.feature_importances_ for tree in trees])
    avg_feature_importance = np.mean(feature_importances, axis=0)
    
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(avg_feature_importance)), avg_feature_importance, color='skyblue')
    plt.xlabel("Feature Index")
    plt.ylabel("Average Feature Importance")
    plt.title("Average Feature Importances Across Trees")
    plt.show()
except AttributeError:
    print("Not all trees have a 'feature_importances_' attribute.")

# -------------------------------
# 9. Plot Distribution of Residuals
# -------------------------------
plt.figure(figsize=(12, 6))
# Plot ensemble residual distribution
sns.histplot(ensemble_residual, color="blue", label="Ensemble Residual", kde=True, stat="density", bins=30)

# Plot residual distributions for a few individual trees (first 3 trees)
for i in [38, 39]:
    sns.histplot(residuals[i], kde=True, stat="density", bins=30, label=f"Tree {i} Residual", alpha=0.6)
    
plt.xlabel("Residual (Prediction - True Value)")
plt.ylabel("Density")
plt.title("Residual Distributions: Ensemble vs. Individual Trees")
plt.legend()
plt.show()

# -------------------------------
# 10. Correlation Heatmap for Tree Predictions
# -------------------------------
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, cmap="coolwarm", square=True, cbar_kws={'label': 'Correlation'})
plt.title("Heatmap of Tree Predictions Correlations")
plt.xlabel("Tree Index")
plt.ylabel("Tree Index")
plt.show()

# -------------------------------
# 11. Correlation Heatmap for Tree Residuals
# -------------------------------
plt.figure(figsize=(8, 6))
sns.heatmap(resid_corr_matrix, cmap="coolwarm", square=True, cbar_kws={'label': 'Correlation'})
plt.title("Heatmap of Tree Residuals Correlations")
plt.xlabel("Tree Index")
plt.ylabel("Tree Index")
plt.show()

# -------------------------------
# 12. Boxplots for Individual Tree MSE and MAE
# -------------------------------
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.boxplot(data=tree_mse)
plt.title("Boxplot of Individual Tree MSE")
plt.xlabel("Trees")
plt.ylabel("MSE")

plt.subplot(1, 2, 2)
sns.boxplot(data=tree_mae)
plt.title("Boxplot of Individual Tree MAE")
plt.xlabel("Trees")
plt.ylabel("MAE")
plt.tight_layout()
plt.show()


In [ ]:
split_counts = [tree.num_splits for tree in dyna._trees]
plt.hist(split_counts, bins=range(0, max(split_counts)+2, 2))
plt.title("Distribution of Splits per Tree")
plt.xlabel("Number of Splits")
plt.ylabel("Count of Trees")
plt.show()


In [ ]:
trees = dyna._trees
predictions = np.array([tree.predict(X_train) for tree in trees])
tree_mse = np.array([mean_squared_error(y_train, predictions[i]) for i in range(len(trees))])
best_tree_idxs = np.argsort(tree_mse)
errors = []
for num_trees in range(2, 79):    
    tree_list = [trees[i] for i in best_tree_idxs][:num_trees]
    predictions_test = np.array([tree.predict(X_test) for tree in tree_list])
    ensemble_pred = np.mean(predictions_test, axis=0)
    ensemble_mse = mean_squared_error(y_test, ensemble_pred)
    ensemble_mae = mean_absolute_error(y_test, ensemble_pred)
    errors.append(ensemble_mae)

# Plot the error as a function of the number of trees
plt.figure(figsize=(8, 5))
plt.plot(range(2, 79), errors, marker='o', color='blue')
plt.xlabel("Number of Trees")
plt.ylabel("Ensemble MAE")
plt.title("Ensemble MAE vs. Number of Trees")
plt.grid(True)

In [ ]:
xgb = XGBRegressor()
print(xgb)